In [1]:
import os
import re
import warnings
from tqdm import tqdm

import numpy as np
import pandas as pd
import torch

from transformers import (
    AutoConfig, AutoTokenizer, 
    AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, 
    AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
)

from datasets import load_from_disk

from src.metrics import rouge_for_batch, f1_score_at_k_for_batch, jaccard_similarity_for_batch

os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings('ignore')

In [2]:
NGPU = torch.cuda.device_count()
NCPU = os.cpu_count()
NGPU, NCPU

(6, 64)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Paths and Names

In [4]:
### paths and names

TRAIN_DATA_PATH = 'data/model_dev/model_dev_v3_train.hf'
EVAL_DATA_PATH = 'data/model_dev/model_dev_v3_eval.hf'

MODEL_CHECKPOINT = '.log/paust_pko_t5_base_v3_run_5/checkpoint-11310'

# Load Model & Tokenizer

In [5]:
config = AutoConfig.from_pretrained(MODEL_CHECKPOINT)

In [6]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT, config=config).to(device)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

# Load Data

In [7]:
train_dataset = load_from_disk(TRAIN_DATA_PATH)
eval_dataset = load_from_disk(EVAL_DATA_PATH)

print(train_dataset)
print(eval_dataset)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 9346
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2337
})


In [8]:
tokenizer.decode(train_dataset['input_ids'][0])

'generate keyphrases: "로페테기가 좋아하는 황희찬, 리즈전 선발 가능성 있어" 복귀전에서 득점을 한 황희찬은 선발 출전을 대기하고 있다.울버햄튼은 19일 오전 0시(한국시간) 영국 울버햄튼에 위치한 몰리뉴 스타디움에서 리즈 유나이티드와 2022-23시즌 잉글리시 프리미어리그(EPL) 28라운드를 치른다. 울버햄튼은 13위, 리즈는 19위에 위치 중이다.영국 \'익스프레스 앤 스타\'는 울버햄튼, 리즈 경기를 프리뷰하면서 "홈 팬들은 시작부터 공격적이고 강렬한 경기를 원한다. 훌렌 로페테기 감독은 때로는 보수적인 방식으로 접근을 했다. 홈 팬들 앞에서는 달라야 한다. 공격진 변화가 예상되는 황희찬이 선발로 나설 수 있다"고 전했다.황희찬은 브루노 라즈 감독 아래에선 벤치 신세였다. 로페테기 감독이 온 후엔 달랐다. 득점 수는 적어도 기동력과 저돌적인 황희찬을 선호했다. 경기력으로 응답했다. 2022 국제축구연맹(FIFA) 카타르 월드컵에서 활약으로 자신감까지 올랐다. 좋은 활약을 이어가던 황희찬은 리버풀전에서 부상을 당해 한동안 빠졌다.재활 기간을 거친 황희찬은 지난 뉴캐슬 유나이티드전에 복귀를 했는데 골을 넣었다. 0-1로 뒤지던 후반 24분 들어와 후반 25분 집중력 있는 모습으로 뉴캐슬 골망을 흔들었다. 경기는 울버햄튼의 1-2 패배로 끝이 났지만 황희찬에겐 고무적인 날이었다. 이날 골로 황희찬은 리그 1호 득점에 성공했고 EPL에선 무려 13개월 만에 골 맛을 봤다. 부상 불운을 골로 보답을 받는 날이었다.\'익스프레스 앤 스타\'는 "로페테기 감독은 황희찬을 정말 좋아한다. 근면한 모습과 라울 히메네스를 대체할 수 있는 능력이 있어 리즈전에 선발로 택할 수 있다. 지난 주말 햄스트링 부상에서 돌아온 황희찬은 뉴캐슬을 상대로 골을 넣었다. 황희찬이 선발로 나오면 레프트백 라얀 아이트-누리와 같이 가능성이 높다"고 구체적인 전망을 내놓았다.울버햄튼 순위는 13위지만 강등권인 18위 본머스와 승점 3점차밖에 안 난다. 리즈전 필승이 요구되는

In [9]:
tokenizer.decode(train_dataset['labels'][0])

'로페테기; 황희찬; 선발 출전; 울버햄튼; 리즈전; EPL; 득점; 복귀전; 한동안; 부상</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

# Functions

In [10]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
    # Replace -100 in the labels as we can't decode them.
    # predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    # labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # ROUGE / Mean Generated Length
    result = rouge_for_batch(decoded_labels, decoded_preds)
    result["gen_len"] = np.mean(prediction_lens)

    # F1@K / Jaccard@K
    result["F1@10"] = f1_score_at_k_for_batch(decoded_labels, decoded_preds, 10)
    result["jaccard_similarity"] = jaccard_similarity_for_batch(decoded_labels, decoded_preds, 10)
    
    result = {key: round(value * 100, 4) for key, value in result.items()}
    
    return result

# Generate

In [11]:
inputs = eval_dataset[:]

input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

In [12]:
### Beam Search Args

generation_args = {
    'num_beams': 5, 
    'no_repeat_ngram_size': 3, 
    'num_return_sequences': 1, 
    'early_stopping': False,
}

generation_args = {
    'do_sample': True, 
    'top_k': 3, 
    'top_p': 0.95, 
}

generation_args = {
    'num_beams': 15, 
    'no_repeat_ngram_size': 3, 
    'num_return_sequences': 1, 
    'num_beam_groups': 3,
    'early_stopping': False,
}

In [21]:
batch_size = 8
start = 0
for idx in tqdm(range(batch_size, 2337+batch_size, batch_size)):
    print(start, idx)
    start = idx

100%|██████████| 293/293 [00:00<00:00, 147089.30it/s]

0 8
8 16
16 24
24 32
32 40
40 48
48 56
56 64
64 72
72 80
80 88
88 96
96 104
104 112
112 120
120 128
128 136
136 144
144 152
152 160
160 168
168 176
176 184
184 192
192 200
200 208
208 216
216 224
224 232
232 240
240 248
248 256
256 264
264 272
272 280
280 288
288 296
296 304
304 312
312 320
320 328
328 336
336 344
344 352
352 360
360 368
368 376
376 384
384 392
392 400
400 408
408 416
416 424
424 432
432 440
440 448
448 456
456 464
464 472
472 480
480 488
488 496
496 504
504 512
512 520
520 528
528 536
536 544
544 552
552 560
560 568
568 576
576 584
584 592
592 600
600 608
608 616
616 624
624 632
632 640
640 648
648 656
656 664
664 672
672 680
680 688
688 696
696 704
704 712
712 720
720 728
728 736
736 744
744 752
752 760
760 768
768 776
776 784
784 792
792 800
800 808
808 816
816 824
824 832
832 840
840 848
848 856
856 864
864 872
872 880
880 888
888 896
896 904
904 912
912 920
920 928
928 936
936 944
944 952
952 960
960 968
968 976
976 984
984 992
992 1000
1000 1008
1008 1016
1016 10

In [13]:
batch_size = 8
predictions = None
with torch.no_grad():
    start = 0
    for idx in tqdm(range(batch_size, len(input_ids)+batch_size, batch_size)):
        ids, mask = input_ids[start:idx], attention_mask[start:idx]
        ids, mask = torch.tensor(ids).to(device), torch.tensor(mask).to(device)

        prediction = model.generate(input_ids=ids, attention_mask=mask, max_length=64, **generation_args)
        if predictions == None:
            predictions = prediction.detach().cpu().tolist()
        else:
            predictions.extend(prediction.detach().cpu().tolist())
        start = idx

predictions_decoded = tokenizer.batch_decode(predictions, skip_special_tokens=True)

100%|██████████| 292/292 [21:35<00:00,  4.44s/it]


In [ ]:
len(predictions)

In [ ]:
len(inputs['labels'])

In [19]:
metrics = compute_metrics((predictions, inputs['labels']))

In [20]:
metrics

{'rouge1': 65.9841,
 'rouge2': 45.8601,
 'rougeL': 53.5776,
 'rougeLsum': 53.5776,
 'gen_len': 100.0,
 'F1@10': 58.2764,
 'jaccard_similarity': 25.382}

```
generation_args = {
    'num_beams': 5, 
    'no_repeat_ngram_size': 3, 
    'num_return_sequences': 1, 
    'early_stopping': False,
}

{'rouge1': 65.9559,
 'rouge2': 45.8405,
 'rougeL': 53.5547,
 'rougeLsum': 53.5547,
 'gen_len': 100.0,
 'F1@10': 58.2764,
 'jaccard_similarity': 25.382}
```

```
generation_args = {
    'do_sample': True, 
    'top_k': 3, 
    'top_p': 0.95, 
}

{'rouge1': 64.5375,
 'rouge2': 43.5099,
 'rougeL': 52.8016,
 'rougeLsum': 52.8016,
 'gen_len': 100.0,
 'F1@10': 58.6438,
 'jaccard_similarity': 24.1092}
```

```
generation_args = {
    'num_beams': 15, 
    'no_repeat_ngram_size': 3, 
    'num_return_sequences': 1, 
    'num_beam_groups': 3,
    'early_stopping': False,
}

{'rouge1': 65.9841,
 'rouge2': 45.8601,
 'rougeL': 53.5776,
 'rougeLsum': 53.5776,
 'gen_len': 100.0,
 'F1@10': 58.2764,
 'jaccard_similarity': 25.382}
```

In [22]:
for data, pred in zip(eval_dataset, predictions):
    pred = np.where(pred != -100, pred, tokenizer.pad_token_id)
    # context = tokenizer.decode(data['input_ids'], skip_special_tokens=True)
    summary = tokenizer.decode(data['labels'], skip_special_tokens=True)
    pred = tokenizer.decode(pred, skip_special_tokens=True)
    # print(f'입력: {context}')
    print(f'정답: {summary}')
    print(f'예측: {pred}', end='\n\n')

정답: 토트넘; 우승; 아스널; 맨시티; 승점 차; 빅4; 브라이튼; 강등 전쟁; 슈퍼컴퓨터; 노팅엄 포레스트
예측: 토트넘; 빅4; 슈퍼컴퓨터; 우승; 브라이턴; 뉴캐슬 유나이티드; 맨유; 승점 차; 강등 전쟁; 레스터시티; 노팅엄 포레스트

정답: 경부고속도; 지하화; 서울 리니어파크; 강남 도심; 상부 공간; 녹지 공간; 연결; 서울시장; 지하도로; 착공
예측: 경부고속도로; 양재~한남; 지하화; 상부공간; 이용방안; 서울 리니어파크; 강남도심; 동서 지역; 리오 공원; 복합문화 공간

정답: 중기부; 중소기업부; 수출활성화; TF(태스크포스); 글로벌 강소기업; 육성; 1000+ 프로젝트; 글로벌 비즈니스센터(GBC); 개편전략; 수출인큐베이터(BI)
예측: 중소벤처기업부; 수출활성화; TF; 민관 협·단체장; 연구기관; 합동 대응체계; 글로벌 강소기업; 지원체계 개편방향; 해외마케팅; 스마트공장

정답: 하태경; 윤석열 대통령; 부산세계박람회; BIE; 일광수산 횟집; 친일몰이; 더탐사; 선라이즈; 조선시대 지명; 괴담 언론
예측: 하태경; 윤석열 대통령; 방문 횟집; 친일몰이; 더탐사; 일광; 욱일기 상징; 조선시대 지명; 언론자격 의문; 부산세계박람회

정답: SSG 추신수; 외야 수비; 1번 우익수; 대전 한화전; 첫 수비 출장; 지명타자; 개막 7경기; 왼쪽 팔꿈치 수술; 스프링캠프; 선발 라인업
예측: SSG 추신수; 외야 수비; 1번타자 우익수; 선발 라인업; 개막 7경기; 첫 수비 출장; 주전 우익수; 출루; 베이스러닝; 시즌 준비 과정; 선수 몸 상태

정답: 전처 남친 살해; 재판; 혐의; 징역 19년; B씨; 이혼; 주거지; 1심; C씨; 대법원
예측: 전처 남친 살해; 30대; 징역 19년 확정; 이혼한 전 부인; 남자친구; 살해한 혐의; 대법원 2부; 살인; C씨; 흉기로 살해

정답: 은행권 발행; 코코본드; 조건부자본증권; 발행 잔액; 상각; 회계상 자본; 상각 사유; 자기자본비율; 투자심리 위축; 금융시장
예측: 국내 은행권